In [1]:
import pandas as pd
import numpy as np
import io
import os
import json
import distutils.dir_util
from collections import Counter
from tensorflow.keras.models import save_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense, SimpleRNN,Dropout

## 데이터 전처리

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [3]:
# train data 준비
f_train = pd.read_json('/gdrive/MyDrive/autoencoder/train.json' ,typ = 'frame', encoding="utf-8")
df_train = pd.DataFrame(f_train)
# df_train = df_train.sort_values(by=['like_cnt'],ascending=False)
# df_train = df_train[df_train['like_cnt']>15] >> local에서 split 했기 때문에 생략
df_train.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[듀엣, 감각적인, 콜라보레이션, 최신, 트렌디, 팝]",25221,이 조합은 믿고 들어야지! 내가 사랑하는 아티스트들의 콜라보!,"[309363, 535261, 459256, 286829, 15124, 222305...",92,2020-04-03 16:57:44.000
1,"[힐링, 휴식, 잔잔한]",75391,"잔잔하고 은은한 노래들, 공부할 때 들으면 좋은 노래들","[281630, 24016, 644788, 71947, 664949, 332442,...",15,2014-08-10 17:22:46.000
2,"[산책, 감성, 좋은노래, 인디, 어쿠스틱]",108360,늦은밤 산책할때 들으면 더 좋아지는 노래들,"[402467, 2718, 44858, 83846, 413444, 547817, 4...",95,2017-11-03 10:32:56.000


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31405 entries, 0 to 31404
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          31405 non-null  object
 1   id            31405 non-null  int64 
 2   plylst_title  31405 non-null  object
 3   songs         31405 non-null  object
 4   like_cnt      31405 non-null  int64 
 5   updt_date     31405 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.4+ MB


In [5]:
# validation data 준비
f_val = pd.read_json('/gdrive/MyDrive/autoencoder/val.json' ,typ = 'frame', encoding="utf-8")
df_val = pd.DataFrame(f_val)
# df_val = df_val.sort_values(by=['like_cnt'],ascending=False)
# # df_val = df_val[df_val['like_cnt']>15]
df_val.head()

,tags,id,plylst_title,songs,like_cnt,updt_date
0,"[클럽, 운동]",146036,헤드셋에서 진동느껴지는 Electronica,"[187020, 472939, 278862, 338876, 74547, 80882,...",232,2016-03-13 23:10:57.000
1,"[헤이즈, 투유프로젝트슈가맨, 원곡모음집, 김이나, 박나래, 레벨조이, 산다라박, ...",22732,JTBC 투유프로젝트 슈가맨 원곡모음zip (상시업뎃~!!),"[219072, 316497, 160099, 145523, 82330, 690251...",196,2020-03-07 00:57:24.000
2,"[달달하게, 음악, 따뜻하게, 봄]",152069,다가오는 봄 다가오는 음악,"[572307, 555386, 132583, 504477, 460503, 15038...",30,2016-05-15 18:00:35.000
3,"[추천곡, 좋은노래, 보석같은곡]",116065,○ 추천^^차트에 없지만 묻히기엔 아까운 100곡(PART 2) ○,"[131901, 4421, 416742, 55202, 688512, 171409, ...",400,2016-12-12 12:33:51.000
4,[팝],150727,평생 잊지 못할 내 머릿 속 명곡 part 1,"[540759, 439961, 247350, 399291, 545274, 41413...",35,2011-02-12 00:30:09.000


In [6]:
df_val.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7852 entries, 0 to 7851
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   tags          7852 non-null   object
 1   id            7852 non-null   int64 
 2   plylst_title  7852 non-null   object
 3   songs         7852 non-null   object
 4   like_cnt      7852 non-null   int64 
 5   updt_date     7852 non-null   object
dtypes: int64(2), object(4)
memory usage: 368.2+ KB


In [7]:
# 플레이리스트 내 song id 리스트
train_song = df_train['songs']
# 플레이리스트 내 tag 리스트
train_tag = df_train['tags']
# 플레이리스트 내 song id 리스트 전체 나열
song_list = [song for plist in train_song for song in plist]
# 플레이리스트 내 tag 리스트 전체 나열
tag_list = [tag for plist in train_tag for tag in plist]

In [8]:
# 전체 나열 리스트 중 각 song의 개수
count_song = Counter(song_list)
# 전체 나열 리스트 중 각 tag의 개수
count_tag = Counter(tag_list)

In [9]:
# 전체 플레이리스트 내 20회 넘게 담긴 song을 dictionary에 추가
mfl={}  # meaningful
for key, value in count_song.items():
    if value>20:
        mfl[key]=value
song_len = len(mfl) # 20회 이상 담긴 song의 개수
song_len

16375

In [10]:
# 전체 플레이리스트 내 10회 넘게 담긴 tag을 dictionary에 추가
for key, value in count_tag.items():
    if value>10:
        mfl[key]=value
tag_len=len(mfl)-song_len # 11회 이상 담긴 tag의 개수
tag_len

1184

In [11]:
mfl_col = list(mfl.keys())
len(mfl_col) # 유의미한 song과 tag의 총 개수 (song_len+tag_len)

17559

In [12]:
# 전체 플레이리스트를 row, 유의미한 song과 tag를 column으로 하는 zero matrix 생성
zero_matrix = np.zeros((len(df_train),len(mfl_col)))
zero_matrix.shape

(31405, 17559)

In [13]:
df_zero = pd.DataFrame(zero_matrix,columns=mfl_col,index=df_train['id'])
df_zero.head() # matrix >> dataframe 변환

,535261,459256,286829,15124,222305,318762,143345,298043,380357,64691,...,따스함,Hot100,다양한,Soundtrack,빅룸,지코,어쿠스틱기타,멜로우힙합,1970년대,5월
id,,,,,,,,,,,,,,,,,,,,,
25221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
108360,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
146788,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# 각 플레이리스트에 해당하는 곡, 태그 좌표에 1 부여

def create_onehot(df,column_name):
    zero_matrix=np.zeros((len(df),len(column_name)))
    zero_df=pd.DataFrame(zero_matrix,columns=column_name,index=df['id'])
    for i in range(len(df)):
        for tag,song in zip(df.iloc[i,0],df.iloc[i,3]):
            if tag in column_name:
                zero_df.iloc[i,column_name.index(tag)]=1
            if song in column_name:
                zero_df.iloc[i,column_name.index(song)]=1
    return zero_df

In [15]:
# create_onehot 함수 활용 train data >> onehot encoding
train_onehot = create_onehot(df_train, mfl_col)
train_onehot.shape

(31405, 17559)

In [16]:
# create_onehot 함수 활용 validation data >> onehot encoding
val_onehot = create_onehot(df_val, mfl_col)
val_onehot.shape

(7852, 17559)

## 오토인코더(학습)

In [42]:
encoding_dim=64
input_pl = Input(shape=(len(mfl_col),)) # one-hot encoding된 DataFrame의 행이 Input
dropout = Dropout(0.2)(input_pl)
encoded = Dense(encoding_dim,activation='relu')(input_pl) # 64개로 축소(인코딩)
encoded = Dense(36,activation='relu')(encoded) # 36개로 축소(인코딩)

decoded = Dense(len(mfl_col),activation='sigmoid')(encoded) # 다시 원래 column 개수로 복원(디코딩)
autoencoder = Model(input_pl,decoded)

autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 21965)]           0         
                                                                 
 dense (Dense)               (None, 64)                1405824   
                                                                 
 dense_1 (Dense)             (None, 36)                2340      
                                                                 
 dense_2 (Dense)             (None, 21965)             812705    
                                                                 
Total params: 2,220,869
Trainable params: 2,220,869
Non-trainable params: 0
_________________________________________________________________


In [43]:
autoencoder.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [44]:
history = autoencoder.fit(train_onehot,train_onehot,epochs=5,batch_size=64,validation_split=0.2,verbose=1) # 기존의 데이터프레임 복원이 목적이기 때문에, test data == train data

Epoch 1/5
393/393 [==============================] - 16s 23ms/step - loss: 0.0960 - accuracy: 0.0033 - val_loss: 0.0031 - val_accuracy: 0.0033
Epoch 2/5
393/393 [==============================] - 6s 16ms/step - loss: 0.0027 - accuracy: 0.0038 - val_loss: 0.0026 - val_accuracy: 0.0033
Epoch 3/5
393/393 [==============================] - 6s 16ms/step - loss: 0.0023 - accuracy: 0.0048 - val_loss: 0.0024 - val_accuracy: 0.0204
Epoch 4/5
393/393 [==============================] - 6s 15ms/step - loss: 0.0022 - accuracy: 0.0121 - val_loss: 0.0023 - val_accuracy: 0.0185
Epoch 5/5
393/393 [==============================] - 6s 16ms/step - loss: 0.0021 - accuracy: 0.0197 - val_loss: 0.0022 - val_accuracy: 0.0180


In [45]:
history = autoencoder.fit(train_onehot,train_onehot,epochs=5,batch_size=64,validation_split=0.2,verbose=1) # 기존의 데이터프레임 복원이 목적이기 때문에, test data == train data

Epoch 1/5
393/393 [==============================] - 8s 21ms/step - loss: 0.0020 - accuracy: 0.0232 - val_loss: 0.0021 - val_accuracy: 0.0191
Epoch 2/5
393/393 [==============================] - 6s 16ms/step - loss: 0.0020 - accuracy: 0.0297 - val_loss: 0.0021 - val_accuracy: 0.0406
Epoch 3/5
393/393 [==============================] - 6s 16ms/step - loss: 0.0020 - accuracy: 0.0399 - val_loss: 0.0021 - val_accuracy: 0.0524
Epoch 4/5
393/393 [==============================] - 6s 16ms/step - loss: 0.0020 - accuracy: 0.0476 - val_loss: 0.0020 - val_accuracy: 0.0522
Epoch 5/5
393/393 [==============================] - 6s 16ms/step - loss: 0.0019 - accuracy: 0.0584 - val_loss: 0.0020 - val_accuracy: 0.0634


In [46]:
predict_plist = autoencoder.predict(val_onehot)

246/246 [==============================] - 1s 3ms/step


In [47]:
df_id = list(df_val['id'])
col= mfl_col
ori_song = col[:song_len]
ori_tag = col[song_len:]

song_predict = predict_plist[:,:song_len] # song output(추천곡)
tag_predict = predict_plist[:,song_len:] # tag output(추천태그)

In [48]:
result=[]
n=0
for i in df_id:
    dic={}
    dic['id']=i

    plist_song=song_predict[n].argsort()[-100:] # predict한 song output 중 상위 100개
    p_song=[]
    for song in plist_song:
        p_song.append(ori_song[song])
    dic['songs']=p_song

    plist_tag=tag_predict[n].argsort()[-10:] # predict한 tag output 중 상위 10개
    p_tag=[]
    for tag in plist_tag:
        p_tag.append(ori_tag[tag])
    dic['tags']=p_tag
    n+=1
    result.append(dic)

In [49]:
def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    parent = os.path.dirname(fname+".json")
    distutils.dir_util.mkpath(parent)
    with io.open( fname+".json", "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [50]:
write_json(result,'/gdrive/MyDrive/autoencoder/results')